In [3]:
#THaW Project
#This program is meant to make a Naive Bayes algorithm to test the data collected by MSU
#6/12/2019
#Code Written by: Manzi Bryan

import math
import pandas as pd
import statistics

path=r'C:\Users\brnma\msudata.xlsx'
df = pd.ExcelFile(path).parse('Sheet1')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary columns
devices = df['Device'].unique()



def probDensity(instance, mean, stdev):
    
    return ((1/(math.sqrt(2*math.pi)*stdev))*(math.exp(-(math.pow((instance-mean),2))/(2*stdev*stdev))))


# def meanAndSD(df):
#     stats = pd.DataFrame(index = ['mean', 'stdev'], columns=df.columns[3:])
    
#     meanList = df.mean(axis=0)[2:]
#     for feat in df.columns[3:]:
#         stats[feat]['mean'] = meanList[feat]

#     stdevList = df.std(axis=0)[2:]
#     for feat in df.columns[3:]:
#         stats[feat]['stdev'] = stdevList[feat]
    
#     return stats

# #meanAndSD(df)


def priorProb(df, device):
    global devices
    values = df['Device'].value_counts()

    i=0
    for d in devices:
        if(d in device or device in d):
            return(values[i])
        else:
            i+=1
#priorProb(df, 'Samsung Hub1')


def caller(df):
    i = 1
    total = 0
    correct = 0
    incorrect = 0
    
    while i <= 10: 
        tt = ninetyTenSplit(df, i)
        i+=1
        ninety = tt['train']
        tenLabeled = tt['testLabeled']
        tenUnlabeled = tt['testUnlabeled']

        score = classifyDevices(ninety, tenLabeled, tenUnlabeled)
        
        correct += score['Correct']
        incorrect += score['Incorrect']
        total += (score['Incorrect'] + score['Correct'])
    print('Program has ' + str((correct/total) * 100)  + '% accuracy')
    print(total)


    

def classifyDevices(ninety, tenLabeled, tenUnlabeled):#Function receives an unlabeled dataframe for training
    stats = meanAndSD(ninety)
    possibleDevices = pd.DataFrame(index = devices, columns=['Probability'])
    means = stats['means']
    stdevs = stats['stdevs']
    
#     print(tenUnlabeled)
#     print(means)

    i = 0
    correct = 0
    incorrect = 0
    
    while i not in tenUnlabeled.index:
        i+=1
        
    while i <= tenUnlabeled.index.max():
        
        for device in devices:
            prob = 1
            for feat in tenUnlabeled.columns[2:]:
                mean = means[feat][device]
                stdev = stdevs[feat][device]
                instance = tenUnlabeled[feat][i]
                prob *= probDensity(instance, mean, stdev)
            possibleDevices['Probability'][device] = prob
        likelyDevice = devices[0]
        
        for dev in devices:
            if possibleDevices['Probability'][dev] > possibleDevices['Probability'][likelyDevice]:
                likelyDevice = dev
                
        actualDevice = tenLabeled['Device'][i]
#         print("actual device : " + str(tenLabeled[feat][i]) + " likely device " + str(tenUnlabeled[feat][i]) )
        if(actualDevice == likelyDevice):
            correct+=1
        else:
            incorrect+=1
        i+=10
    score = {}
    score['Total'] = len(tenUnlabeled.index)
    score['Correct'] = correct
    score['Incorrect'] = incorrect
    print(str(correct) + " correct and " + str(incorrect) + " incorrect and " + str(score['Total']) + " total")
    return score
                
    
        
    
def ninetyTenSplit(df, remove):
    ninety = pd.DataFrame(index = range(0, math.ceil(0.9*(df.index.max()))), columns=df.columns)
    ten = pd.DataFrame(index = range(0, math.ceil(0.1*(df.index.max()))), columns=df.columns)
    
    ninety = df
    ten = df
    i = 0
    
    while i <= df.index.max():
        if (i-remove)%10 == 0:
            ninety = ninety[ninety.index != i]
        else:
            ten = ten[ten.index != i]
        i+=1
    
    trainAndTest = {}
    trainAndTest['train'] = ninety
    trainAndTest['testLabeled'] = ten
    copyTen = ten.copy(deep=True)
    copyTen.drop(copyTen.columns[copyTen.columns.str.contains('Device',case = False)],axis = 1, inplace = True)
    trainAndTest['testUnlabeled'] = copyTen
    
#     print(copyTen)
    return trainAndTest
# ninetyTenSplit(df, 10)
            
            

# def meanDevices(df):
#     global devices
#     #Should be a table showing the mean values for all of the devices and their respective features:
    
#     numDevices = len(devices)
#     means = pd.DataFrame(index = devices, columns=df.columns[3:])
    
#     for feat in  means.columns:
#         i = 0
#         count = 0
#         for device in means.index:
#             total = 0
#             count = 0
#             while i <= df.index.max() and df['Device'][i] == device:
#                 total+=float(df[feat][i])
#                 i+=1    
#                 count+=1
#             means[feat][device] = total/count
#     print(means)
#     return means
# meanDevices(df)

def meanAndSD(df):
    global devices
    
    numDevices = len(devices)
    stdevs = pd.DataFrame(index = devices, columns=df.columns[3:])
    means = pd.DataFrame(index = devices, columns=df.columns[3:])
    
    for feat in stdevs.columns:
        i = 0
        stdevList = []
        meansList = []
        
        if i not in df.index:
            i+=1
        device = df['Device'][i]
        while i <= df.index.max():
            
            if df['Device'][i] == device:
                stdevList.append(df[feat][i])
                meansList.append(df[feat][i])
                i+=1
                if i not in df.index and i < df.index.max():
                    i+=1
            else:  
               
                stdevs[feat][device] = statistics.stdev(stdevList)
                means[feat][device] = statistics.mean(meansList)
                
                stdevList = []
                meansList = []
                device = df['Device'][i]
    
        stdevs[feat][device] = statistics.stdev(stdevList)
        means[feat][device] = statistics.mean(meansList)
        
    stats = {}
    stats['means'] = means
    stats['stdevs'] = stdevs
    
    return stats

# meanAndSD(df)  

caller(df)


12 correct and 47 incorrect and 60 total
15 correct and 44 incorrect and 60 total
10 correct and 49 incorrect and 60 total
14 correct and 45 incorrect and 60 total
10 correct and 49 incorrect and 60 total
10 correct and 49 incorrect and 60 total
12 correct and 47 incorrect and 60 total
13 correct and 46 incorrect and 60 total
15 correct and 44 incorrect and 60 total
11 correct and 48 incorrect and 60 total
Program has 20.677966101694913% accuracy
590


In [11]:
%autosave 20

Autosaving every 20 seconds
